# Creating a Median Composite with Dask

We will query a STAC catalog for Sentinel-2 imagery and create a monthly cloud-free composite using distributed processing on a local machine.

## Setup and Data Download

The following blocks of code will install the required packages and download the datasets to your Colab environment.

In [12]:
%%capture
if 'google.colab' in str(get_ipython()):
    !pip install pystac-client
    !apt install libspatialindex-dev
    !pip install fiona shapely pyproj rtree
    !pip install geopandas folium stackstac rioxarray mapclassify

In [13]:
import json
import geopandas as gpd
from shapely.geometry import mapping
import pandas as pd
import pystac_client
import os
import folium
from folium import Figure
import stackstac
import rioxarray
import matplotlib.pyplot as plt
import mapclassify
import dask

In [14]:
from dask.distributed import Client, progress
client = Client()  # set up local cluster on the machine
client

C:\Users\ujava\anaconda3\envs\eocloud\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55183 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:55183/status,
Dashboard: http://127.0.0.1:55183/status,Workers: 4
Total threads: 8,Total memory: 15.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55184,Workers: 4
Dashboard: http://127.0.0.1:55183/status,Total threads: 8
Started: Just now,Total memory: 15.79 GiB
Comm: tcp://127.0.0.1:55203,Total threads: 2
Dashboard: http://127.0.0.1:55208/status,Memory: 3.95 GiB
Nanny: tcp://127.0.0.1:55187,


In [15]:
data_folder = 'data'
output_folder = 'output'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [16]:
def download(url):
    filename = os.path.join(data_folder, os.path.basename(url))
    if not os.path.exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/spatialthoughts/python-tutorials/raw/main/data/' +
         'bangalore.geojson')

## Procedure

Let's use Element84 search endpoint to look for items from the `sentinel-2-c1-l2a` collection on AWS

In [17]:
catalog = pystac_client.Client.open('https://earth-search.aws.element84.com/v1')

In [18]:
aoi_file = 'bangalore.geojson'
aoi_filepath = os.path.join(data_folder, aoi_file)
aoi = gpd.read_file(aoi_filepath)

In [19]:
geometry = aoi.unary_union
geometry_geojson = json.dumps(mapping(geometry))

We search for the imagery collected within the date range and intersecting the AOI geometry. Additionally we add filters to select imagery with less cloud cover and over a specific MGRS tile.

In [20]:
year = 2023
month = 5
time_range = f'{year}-{month:02}'

search = catalog.search(
    collections=['sentinel-2-c1-l2a'],
    intersects=geometry_geojson,
    datetime=time_range,
    query={'eo:cloud_cover': {'lt': 30},  'mgrs:grid_square': {'eq': 'GQ'}},
)
items = search.item_collection()
len(items)

3

In [21]:
stack = stackstac.stack(items, resolution=10)
stack

C:\Users\ujava\anaconda3\envs\eocloud\Lib\site-packages\stackstac\prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-a95fee2162b54c6224f5f427b95a6391' (time: 3,
                                                                band: 19,
                                                                y: 11179,
                                                                x: 11184)> Size: 57GB
dask.array<fetch_raster_window, shape=(3, 19, 11179, 11184), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/59)
  * time                                     (time) datetime64[ns] 24B 2023-0...
    id                                       (time) <U30 360B 'S2B_T43PGQ_202...
  * band                                     (band) <U8 608B 'red' ... 'preview'
  * x                                        (x) float64 89kB 6.992e+05 ... 8...
  * y                                        (y) float64 89kB 1.501e+06 ... 1...
    grid:code                                <U10 40B 'MGRS-43PGQ'
    ...                                       ...
    file:checksum                            (band) object 152B None ... None
    file:size                                (band) object 152B None ... None
    common_name                              (band) object 152B 'red' ... None
    center_wavelength                        (band) object 152B 0.665 ... None
    full_width_half_max                      (band) object 152B 0.038 ... None
    epsg                                     int32 4B 32643
Attributes:
    spec:        RasterSpec(epsg=32643, bounds=(699160, 1389220, 811000, 1501...
    crs:         epsg:32643
    transform:   | 10.00, 0.00, 699160.00|\n| 0.00,-10.00, 1501010.00|\n| 0.0...
    resolution:  10

Clip and select the subset of bands.

In [29]:
geometry = aoi.to_crs(stack.rio.crs).geometry
clipped = stack.rio.clip(geometry)
subset = clipped.sel(band=['red', 'green', 'blue', 'scl'])

In [30]:
scl = subset.sel(band='scl')

valid = ((scl >= 4) & (scl <= 7) | (scl==11))
         
subset_masked = subset.where(valid)
rgb_masked = subset_masked.sel(band=['red', 'green', 'blue'])

In [31]:
median = rgb_masked.median(dim='time')
median

<xarray.DataArray 'stackstac-a95fee2162b54c6224f5f427b95a6391' (band: 3,
                                                                y: 3427, x: 3510)> Size: 289MB
dask.array<nanmedian, shape=(3, 3427, 3510), dtype=float64, chunksize=(1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/23)
  * band                                     (band) <U8 96B 'red' 'green' 'blue'
  * x                                        (x) float64 28kB 7.67e+05 ... 8....
  * y                                        (y) float64 27kB 1.454e+06 ... 1...
    grid:code                                <U10 40B 'MGRS-43PGQ'
    processing:software                      object 8B {'sentinel-2-c1-l2a-to...
    mgrs:utm_zone                            int32 4B 43
    ...                                       ...
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    s2:snow_ice_percentage                   int32 4B 0
    file:checksum                            (band) object 24B None None None
    file:size                                (band) object 24B None None None
    epsg                                     int32 4B 32643
    spatial_ref                              int32 4B 0

In [32]:
%time median = median.compute()

C:\Users\ujava\anaconda3\envs\eocloud\Lib\site-packages\distributed\client.py:3161: UserWarning: Sending large graph of size 119.35 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


CPU times: total: 36.3 s
Wall time: 6min 15s


In [33]:
output_file = f'median_{year}_{month:02}.tif'
output_path = os.path.join(output_folder, output_file)
median.rio.to_raster(output_path, driver='COG')
print(f'Wrote {output_file}')

Wrote median_2023_05.tif
